In [1]:
with open("mod_hyde/data/all_book_data.txt","r") as f:
    all_books_data = f.read() 

In [2]:
import json

youtube_transcripts_list = ["chunked_misc_transcripts.json","chunked_transcripts_undergrad.json","chunked_transcripts_mba.json"]    
all_youtube_data = []
for file in youtube_transcripts_list:
    with open('mod_hyde/data/YouTube_API_Transcripts/'+file) as f:
        data = json.load(f)
    for id,data in data.items():
        for d in data:
            all_youtube_data.append(d['text'])

In [3]:
import re

all_books_data_list = re.split(r"(?<=[.?!])\s+", all_books_data)

## CHUNKING 

In [2]:
!pip install -Uqq langchain

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    separators = [". "],
    chunk_size=1750,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,

)

books_data_splitted_text = text_splitter.split_text(all_books_data)

In [5]:
all_data = books_data_splitted_text + all_youtube_data

## DISTILBERT

In [7]:
from transformers import AutoTokenizer

import pandas as pd

df = pd.DataFrame(all_data,columns=["text"])

def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
from datasets import Dataset
import pandas as pd

hf_dataset = Dataset.from_pandas(df)

tokenized_datasets = hf_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

/home/athekunal/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Map (num_proc=4): 100%|██████████| 8394/8394 [00:03<00:00, 2659.38 examples/s]


In [8]:
block_size = 1024
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)


Map (num_proc=4): 100%|██████████| 8394/8394 [00:10<00:00, 808.40 examples/s] 


In [18]:
# lm_datasets[0]['input_ids']

In [17]:
lm_datasets[0]['labels'] == lm_datasets[0]['input_ids']

True

In [6]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# model_name = "distilbert/distilgpt2"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

In [21]:
all_books_data_list[1]

'These stories not only sound persuasive and reasonable but are also backed up by evidence - anecdotal, in some cases, and statistical, in others - that the strategies work.'

In [1]:
import pandas as pd

wiki_df = pd.read_csv("mod_hyde/data/wikipedia_data.csv")

In [3]:
wiki_df.isna().sum()

title       0
text     4332
dtype: int64

In [4]:
val_wiki_df = pd.read_csv("mod_hyde/data/wikipedia_data_val.csv") 

In [5]:
val_wiki_df.isna().sum()

title       0
text     6390
dtype: int64

In [16]:
val_wiki_df['len_count'] = val_wiki_df.apply(lambda row: len(str(row.text).split(" ")),axis=1)

In [17]:
def gte_100_words(row):
    if row['len_count'] >= 100:
        return True
    else:
        return False

val_wiki_df = val_wiki_df[val_wiki_df.apply(gte_100_words,axis=1)]
len(val_wiki_df)

45252

In [18]:
from datasets import load_dataset
hotpot_qa = load_dataset("hotpot_qa","distractor")

/home/athekunal/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [43]:
shuffled_dataset = hotpot_qa['train'].shuffle(42)
subset_data = shuffled_dataset[0:5000]

In [44]:
val_subset = subset_data['supporting_facts']
titles = []
for sd in val_subset:
    titles.append(sd['title'])

In [45]:
val_questions = subset_data['question']

In [46]:
val_wiki_list = list(wiki_df['title'].values)
question_not_there = []
question_ther = []
for idx,title in enumerate(titles):
    for t in title:
        if t not in val_wiki_list:
            question_not_there.append(idx)
            continue

In [47]:
len(val_questions) - len(question_not_there)

2662

In [48]:
len(question_not_there)

2338

In [66]:
question_there_idx = [titles[id] for id in range(len(val_questions)) if id not in question_not_there]

In [69]:
wiki_df.head()

,title,text,len_count
0,"Vinalhaven, Maine",Archeological remains indicate that the island...,996
1,"Owls Head, Maine",When Samuel de Champlain explored Owl's Head i...,347
2,"North Haven, Maine","As early as 3300 BCE, the island was visited b...",515
3,Downeast Flight 46,There was poor visibility around Rockland on t...,617
5,Sacramento International Airport,Sacramento International Airport (SMF) opened ...,1260


In [87]:
final_df = pd.DataFrame(columns=['title','text'])
all_titles = []
all_texts = []
for t in question_there_idx[:1000]:
    for wikipedia_titles in t:
        all_titles.append(wikipedia_titles)
        all_texts.append(wiki_df[wiki_df['title'] == wikipedia_titles]['text'].values[0])

final_df['title'] = all_titles
final_df['text'] = all_texts

In [92]:
from langchain_community.document_loaders import WikipediaLoader
import wikipediaapi
import backoff
import requests
from tqdm import tqdm

wikipedia_module = wikipediaapi.Wikipedia("HyDE-Project (athekunal@gmail.com)", "en")

@backoff.on_exception(backoff.expo,requests.exceptions.Timeout,max_tries=5,max_time=30)
def get_wikipedia(title):
    page_py = wikipedia_module.page(title)
    title_text = ""
    for st in page_py.sections:
        if st.title.lower() == 'references' or st.title.lower() == 'external links' or st.title == "References" or st.title =="External link": 
            continue
        title_text += st.text + "\n"
    return title_text

from tqdm import tqdm
import os
import pandas as pd

def is_file_empty(file_path):
    return os.path.getsize(file_path) == 0

if not os.path.exists(f"wikipedia_data_val.csv"):
    df = pd.DataFrame(columns=["title","text"],index=None)
    df.to_csv(f"wikipedia_data_val.csv",index=False,header=True)
else:
    df = pd.read_csv(f"wikipedia_data_val.csv",index_col=False)

if not os.path.exists(f"done_wikipedia_val.txt"):
    os.mknod(f"done_wikipedia_val.txt")
for titles in question_there_idx[:1000]
    for title in tqdm(titles):
        # docs = WikipediaLoader(
        #         query=title, load_max_docs=1, doc_content_chars_max=1
        #     ).load()
        # wikipedia_title = docs[0].metadata["title"]
        if not is_file_empty(f"done_wikipedia_val.txt"):
            with open(f"done_wikipedia_val.txt","r") as f:
                done_titles = [x for x in f.read().splitlines()]
            # print(done_pids)
            if title in done_titles: 
                continue
        title_text = get_wikipedia(title)
        df_dict = {
            "title":[title],
            "text":[title_text]
        }
        df = pd.DataFrame(df_dict)
        
        df.to_csv(f"wikipedia_data_val.csv", mode='a',index=False,header=False)
        with open(f"done_wikipedia_val.txt", "a") as f:
            f.write(f"{title}\n")

['Peter Hobbs (engineer)', 'Russell Hobbs']

In [89]:
final_df.drop_duplicates(subset=['title'],inplace=True)

len(final_df)

1977

In [90]:
train_data_wikipedia_text = list(final_df['text'].values)
with open("mod_hyde/data/wikipedia.txt",'w') as f:
    for line in train_data_wikipedia_text:
        f.writelines(line)

In [25]:

not_there = 0
for title in titles:
    if title not in val_wiki_list:
        not_there += 1

In [27]:
len(titles)

18005

In [8]:
wiki_df['len_count'] = wiki_df.apply(lambda row: len(str(row.text).split(" ")),axis=1)

In [11]:
wiki_df['len_count'].value_counts(sort='asc')

len_count
1       6262
3        711
5        221
7        204
6        193
        ... 
2777       1
3325       1
3898       1
2346       1
2138       1
Name: count, Length: 3222, dtype: int64

In [12]:
wiki_df

,title,text,len_count
0,"Vinalhaven, Maine",Archeological remains indicate that the island...,996
1,"Owls Head, Maine",When Samuel de Champlain explored Owl's Head i...,347
2,"North Haven, Maine","As early as 3300 BCE, the island was visited b...",515
3,Downeast Flight 46,There was poor visibility around Rockland on t...,617
4,Northern California TRACON,Southern California TRACON\n\n\n== References ...,5
...,...,...,...
46306,Dance in California,\n\n\nCrockett-Deane Ballet\nDance Connection ...,40
46307,Miguel de la Madrid,Miguel de la Madrid was born in the city of Co...,1481
46308,Los Pinos,"After the Spanish Conquest, around 1550, a tra...",199
46309,Eugene Allen Gilmore,"Gilmore was born in Brownville, Nebraska to An...",156


In [13]:
def gte_100_words(row):
    if row['len_count'] >= 100:
        return True
    else:
        return False

wiki_df = wiki_df[wiki_df.apply(gte_100_words,axis=1)]

In [14]:
len(wiki_df)

31708

In [15]:
wiki_df

,title,text,len_count
0,"Vinalhaven, Maine",Archeological remains indicate that the island...,996
1,"Owls Head, Maine",When Samuel de Champlain explored Owl's Head i...,347
2,"North Haven, Maine","As early as 3300 BCE, the island was visited b...",515
3,Downeast Flight 46,There was poor visibility around Rockland on t...,617
5,Sacramento International Airport,Sacramento International Airport (SMF) opened ...,1260
...,...,...,...
46305,"Duke Ellington at Fargo, 1940 Live","In 1939, two cooperative extension service wor...",887
46307,Miguel de la Madrid,Miguel de la Madrid was born in the city of Co...,1481
46308,Los Pinos,"After the Spanish Conquest, around 1550, a tra...",199
46309,Eugene Allen Gilmore,"Gilmore was born in Brownville, Nebraska to An...",156


In [49]:
len(wiki_df)

31708